# NGS pipeline for Alzheimer's Disease RNA-seq data

For this project I have used only one control and one sample of someone of advanced AD.

In [44]:
import os
import pandas
from IPython.display import FileLink

Set up some global variables:

In [9]:
# The git dir
os.environ['DIR'] = '/home/mees/NGS_Alzheimer/'
# Working directory to the sequencing data (I use an external disk, since I dont have a lot of space left on the pc)
os.environ['WORKDIR'] = '/media/mees/Elements/ngs_data/'
# The directory to the downloaded genome
os.environ['GENOMEDIR'] = '/media/mees/Elements/ngs_data/genomes/'
# Location to save the indexed reference genome
os.environ['REF_GENOME'] = '/media/mees/Elements/ngs_data/genomes/ref_genome/'

First download the human genome for reference and save it as genome.fa in $GENOMEDIR

In [ ]:
!wget ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
!gzip -d Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
!mv Homo_sapiens.GRCh38.dna.primary_assembly.fa $GENOMEDIR/genome.fa

Download SRA data

In [8]:
!mkdir -p $WORKDIR
!./src/download_data.sh -d $WORKDIR


2020-06-26T09:48:56 prefetch.2.10.7 int: buffer insufficient while reading uri within cloud module - cannot Get Cloud Location
2020-06-26T09:48:57 prefetch.2.10.7: 1) 'SRR6145423' is found locally
2020-06-26T09:48:57 prefetch.2.10.7: 'SRR6145423' has 0 unresolved dependencies

2020-06-26T09:48:57 prefetch.2.10.7 int: buffer insufficient while reading uri within cloud module - cannot Get Cloud Location
2020-06-26T09:48:57 prefetch.2.10.7: 2) 'SRR6145424' is found locally
2020-06-26T09:48:57 prefetch.2.10.7: 'SRR6145424' has 0 unresolved dependencies

2020-06-26T09:48:57 prefetch.2.10.7 int: buffer insufficient while reading uri within cloud module - cannot Get Cloud Location
2020-06-26T09:48:59 prefetch.2.10.7: 3) 'SRR6145425' is found locally
2020-06-26T09:48:59 prefetch.2.10.7: 'SRR6145425' has 0 unresolved dependencies

2020-06-26T09:48:59 prefetch.2.10.7 int: buffer insufficient while reading uri within cloud module - cannot Get Cloud Location
2020-06-26T09:49:00 prefetch.2.10.7: 4

In [12]:
!mkdir -p data/FastQC

After downloading the data and convert the .sra files to .fastq files, use FastQC to determine read quality.

In [31]:
!./src/fastqc.sh -d $WORKDIR 

Started analysis of SRR6145423.sra.fastq
Approx 5% complete for SRR6145423.sra.fastq
Approx 10% complete for SRR6145423.sra.fastq
Approx 15% complete for SRR6145423.sra.fastq
Approx 20% complete for SRR6145423.sra.fastq
Approx 25% complete for SRR6145423.sra.fastq
Approx 30% complete for SRR6145423.sra.fastq
Approx 35% complete for SRR6145423.sra.fastq
Approx 40% complete for SRR6145423.sra.fastq
Approx 45% complete for SRR6145423.sra.fastq
Approx 50% complete for SRR6145423.sra.fastq
Approx 55% complete for SRR6145423.sra.fastq
Approx 60% complete for SRR6145423.sra.fastq
Approx 65% complete for SRR6145423.sra.fastq
Approx 70% complete for SRR6145423.sra.fastq
Approx 75% complete for SRR6145423.sra.fastq
Approx 80% complete for SRR6145423.sra.fastq
Approx 85% complete for SRR6145423.sra.fastq
Approx 90% complete for SRR6145423.sra.fastq
Approx 95% complete for SRR6145423.sra.fastq
Analysis complete for SRR6145423.sra.fastq
'/media/mees/Elements/ngs_data//SRR6145423/FastQC/SRR6145423.s

Approx 95% complete for SRR6145434.sra.fastq
Analysis complete for SRR6145434.sra.fastq
'/media/mees/Elements/ngs_data//SRR6145434/FastQC/SRR6145434.sra_fastqc.zip' -> '/home/mees/NGS_Alzheimer/data/FastQC/SRR6145434.sra_fastqc.zip'
'/media/mees/Elements/ngs_data//SRR6145434/FastQC/SRR6145434.sra_fastqc.html' -> '/home/mees/NGS_Alzheimer/data/FastQC/SRR6145434.sra_fastqc.html'
Started analysis of SRR6145435.sra.fastq
Approx 5% complete for SRR6145435.sra.fastq
Approx 10% complete for SRR6145435.sra.fastq
Approx 15% complete for SRR6145435.sra.fastq
Approx 20% complete for SRR6145435.sra.fastq
Approx 25% complete for SRR6145435.sra.fastq
Approx 30% complete for SRR6145435.sra.fastq
Approx 35% complete for SRR6145435.sra.fastq
Approx 40% complete for SRR6145435.sra.fastq
Approx 45% complete for SRR6145435.sra.fastq
Approx 50% complete for SRR6145435.sra.fastq
Approx 55% complete for SRR6145435.sra.fastq
Approx 60% complete for SRR6145435.sra.fastq
Approx 65% complete for SRR6145435.sra.f

I like to open the created reports of fastq to see if there are some abnormalities.

In [52]:
for file in os.listdir(fastqc_dir):
    if file.endswith(".html"):
        html_file = FileLink(os.path.join('data/FastQC', file))
        display(html_file)

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145434.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145433.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145425.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145427.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145426.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145424.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145435.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145436.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145423.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145437.sra_fastqc.html

FastQC shows that there are Truseq adapter index 7 and NNNNNNN sequences.
We remove these sequences using cutadapt.

In [43]:
#!./src/trim.sh -d $WORKDIR

This is cutadapt 1.15 with Python 3.6.9
Command line parameters: -a NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN -a GATCGGAAGAGCACACGTCTGAACTCCAGTCACGAGATTCCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAA -a GATCGGAAGAGCACACGTCTGAACTCCAGTCACGAGATTCCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAT -o trimmed/SRR6145423.sra.fastq SRR6145423.sra.fastq
Running on 1 core
Trimming 3 adapters with at most 10.0% errors in single-end mode ...
Finished in 1383.31 s (37 us/read; 1.63 M reads/minute).

=== Summary ===

Total reads processed:              37,552,625
Reads with adapters:                37,332,522 (99.4%)
Reads written (passing filters):    37,552,625 (100.0%)

Total basepairs processed: 2,785,289,949 bp
Total written (filtered):      7,860,791 bp (0.3%)

=== Adapter 1 ===

Sequence: NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN; Type: regular 3'; Length: 35; Trimmed: 36966506 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20-29 bp: 2; 30-35 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 0.0%
  G: 0.0%
  T: 0.0%


FastQC analysis after trimming the adapters.

In [44]:
#!./src/fastqc.sh -d $WORKDIR/trimmed/

Started analysis of SRR6145423.sra.fastq
Approx 5% complete for SRR6145423.sra.fastq
Approx 10% complete for SRR6145423.sra.fastq
Approx 15% complete for SRR6145423.sra.fastq
Approx 20% complete for SRR6145423.sra.fastq
Approx 25% complete for SRR6145423.sra.fastq
Approx 30% complete for SRR6145423.sra.fastq
Approx 35% complete for SRR6145423.sra.fastq
Approx 40% complete for SRR6145423.sra.fastq
Approx 45% complete for SRR6145423.sra.fastq
Approx 50% complete for SRR6145423.sra.fastq
Approx 55% complete for SRR6145423.sra.fastq
Approx 60% complete for SRR6145423.sra.fastq
Approx 65% complete for SRR6145423.sra.fastq
Approx 70% complete for SRR6145423.sra.fastq
Approx 75% complete for SRR6145423.sra.fastq
Approx 80% complete for SRR6145423.sra.fastq
Approx 85% complete for SRR6145423.sra.fastq
Approx 90% complete for SRR6145423.sra.fastq
Approx 95% complete for SRR6145423.sra.fastq
Analysis complete for SRR6145423.sra.fastq
Started analysis of SRR6145433.sra.fastq
Approx 5% complete fo

In [45]:
#!firefox $WORKDIR/trimmed/*.html

Now that we have improved the read qualities, we can align the reads to the human reference genome. For this alignment, we use Hisat2.

First thing is to build the reference genome using hisat2-build.
Then we perform the alignment using hisat2.

In [11]:
# Commented so I don't accidently run it again
#!./src/build.sh -g $GENOMEDIR/genome.fa -o $REF_GENOME

Settings:
  Output files: "/media/mees/Elements/ngs_data/genomes/ref_genome//ref_genome.*.ht2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /media/mees/Elements/ngs_data/genomes//genome.fa
Reading reference sizes
  Time reading reference sizes: 00:01:30
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:01:29
  Time to read SNPs and splice sites: 00:00:00
Using parameters --bmax 138086675 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing w

  Sorting block time: 00:00:15
Returning block of 30670784 for bucket 11
  bucket 14: 10%
Getting block 15 of 32
  Reserving size (138086675) for bucket 15
  Calculating Z arrays for bucket 15
  Entering block accumulator loop for bucket 15:
  bucket 13: 100%
  Sorting block of length 62107305 for bucket 13
  (Using difference cover)
  bucket 14: 20%
  bucket 15: 10%
  bucket 14: 30%
  bucket 15: 20%
  bucket 14: 40%
  bucket 15: 30%
  bucket 14: 50%
  bucket 15: 40%
  bucket 15: 50%
  bucket 14: 60%
  bucket 15: 60%
  bucket 14: 70%
  Sorting block time: 00:00:34
Returning block of 62107306 for bucket 13
  bucket 15: 70%
  bucket 14: 80%
Getting block 16 of 32
  Reserving size (138086675) for bucket 16
  Calculating Z arrays for bucket 16
  Entering block accumulator loop for bucket 16:
  bucket 15: 80%
  bucket 14: 90%
  bucket 16: 10%
  bucket 15: 90%
  bucket 16: 20%
  bucket 14: 100%
  Sorting block of length 139760453 for bucket 14
  (Using difference cover)
  bucket 15: 100%
  S

  bucket 31: 90%
  bucket 31: 100%
  Sorting block of length 116855486 for bucket 31
  (Using difference cover)
  Sorting block time: 00:00:32
Returning block of 47458124 for bucket 30
Getting block 32 of 32
  Reserving size (138086675) for bucket 32
  Calculating Z arrays for bucket 32
  Entering block accumulator loop for bucket 32:
  bucket 32: 10%
  bucket 32: 20%
  bucket 32: 30%
  bucket 32: 40%
  bucket 32: 50%
  bucket 32: 60%
  bucket 32: 70%
  bucket 32: 80%
  Sorting block time: 00:01:08
Returning block of 113546999 for bucket 28
  bucket 32: 90%
  bucket 32: 100%
  Sorting block of length 45807214 for bucket 32
  (Using difference cover)
  Sorting block time: 00:01:05
Returning block of 114596170 for bucket 29
  Sorting block time: 00:01:08
Returning block of 116855487 for bucket 31
  Sorting block time: 00:00:22
Returning block of 45807215 for bucket 32
Exited GFM loop
fchr[A]: 0
fchr[C]: 869653843
fchr[G]: 1470243264
fchr[T]: 2073417374
fchr[$]: 2945849067
Exiting GFM::bu

Now we can align our data to the indexed human genome.
First, align without trimming.

In [55]:
!./src/align.sh -g $REF_GENOME -f $WORKDIR/

Hisat2 is aligning SRR6145423 to the reference genome.
37552625 reads; of these:
  37552625 (100.00%) were unpaired; of these:
    2360539 (6.29%) aligned 0 times
    31238345 (83.19%) aligned exactly 1 time
    3953741 (10.53%) aligned >1 times
93.71% overall alignment rate
Hisat2 is aligning SRR6145424 to the reference genome.
38052606 reads; of these:
  38052606 (100.00%) were unpaired; of these:
    3423266 (9.00%) aligned 0 times
    28619217 (75.21%) aligned exactly 1 time
    6010123 (15.79%) aligned >1 times
91.00% overall alignment rate
Hisat2 is aligning SRR6145425 to the reference genome.
36212177 reads; of these:
  36212177 (100.00%) were unpaired; of these:
    2478081 (6.84%) aligned 0 times
    30107607 (83.14%) aligned exactly 1 time
    3626489 (10.01%) aligned >1 times
93.16% overall alignment rate
Hisat2 is aligning SRR6145426 to the reference genome.
42174836 reads; of these:
  42174836 (100.00%) were unpaired; of these:
    2849120 (6.76%) aligned 0 times
    34036

Then align with trimming.

In [47]:
#!./src/align.sh -g $REF_GENOME -f $WORKDIR/trimmed/

Hisat2 is aligning SRR6145423.sra.fastq
--read-lengths arg must be at least 20
HISAT2 version 2.2.0 by Daehwan Kim (infphilo@gmail.com, www.ccb.jhu.edu/people/infphilo)
Usage: 
  hisat2 [options]* -x <ht2-idx> {-1 <m1> -2 <m2> | -U <r> | --sra-acc <SRA accession number>} [-S <sam>]

  <ht2-idx>  Index filename prefix (minus trailing .X.ht2).
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <SRA accession number>        Comma-separated list of SRA accession numbers, e.g. --sra-acc SRR353653,SRR353654.
  <sam>      File for SAM output (default: stdout)

  <m1>, <m2>, <r> can be comma-separated lists (no whitespace) and can be
  specified many ti

  --met <int>           report internal counters & metrics every <int> secs (1)
  --no-head             suppress header lines, i.e. lines starting with @
  --no-sq               suppress @SQ header lines
  --rg-id <text>        set read group id, reflected in @RG line and RG:Z: opt field
  --rg <text>           add <text> ("lab:value") to @RG line of SAM header.
                        Note: @RG line only printed when --rg-id is set.
  --omit-sec-seq        put '*' in SEQ and QUAL fields for secondary alignments.

 Performance:
  -o/--offrate <int> override offrate of index; must be >= index's offrate
  -p/--threads <int> number of alignment threads to launch (1)
  --reorder          force SAM output order to match order of input reads
  --mm               use memory-mapped I/O for index; many 'hisat2's can share

 Other:
  --qc-filter        filter out reads that are bad according to QSEQ filter
  --seed <int>       seed for random number generator (0)
  --non-deterministic seed rand.

In order to view the alignment in IGV, the created sam files should processed.

1. First, the files must be converted to bam files.
2. Second, the bam files must be sorted.
3. Third, the sorted bam files must be indexed.

In [58]:
!./src/samtobam.sh -d $WORKDIR

Samtools is converting SRR6145423.sam to a bam file.
Samtools is sorting SRR6145423.bam
[bam_sort_core] merging from 15 files and 1 in-memory blocks...
Samtools is indexing SRR6145423.sorted.bam
Samtools is converting SRR6145424.sam to a bam file.
Samtools is sorting SRR6145424.bam
[bam_sort_core] merging from 18 files and 1 in-memory blocks...
Samtools is indexing SRR6145424.sorted.bam
Samtools is converting SRR6145425.sam to a bam file.
Samtools is sorting SRR6145425.bam
[bam_sort_core] merging from 14 files and 1 in-memory blocks...
Samtools is indexing SRR6145425.sorted.bam
Samtools is converting SRR6145426.sam to a bam file.
Samtools is sorting SRR6145426.bam
[bam_sort_core] merging from 18 files and 1 in-memory blocks...
Samtools is indexing SRR6145426.sorted.bam
Samtools is converting SRR6145427.sam to a bam file.
Samtools is sorting SRR6145427.bam
[bam_sort_core] merging from 17 files and 1 in-memory blocks...
Samtools is indexing SRR6145427.sorted.bam
Samtools is converting SR

Now that the alignment is finished, we allign reads to exons and genes using HTSeq.

First we need the human .gtf file, which we download via the ensemble ftp server.
Then..

In [4]:
!wget ftp://ftp.ensembl.org/pub/release-100/gtf/homo_sapiens/Homo_sapiens.GRCh38.100.gtf.gz
!gzip -d Homo_sapiens.GRCh38.100.gtf.gz
!mv Homo_sapiens.GRCh38.100.gtf $GENOMEDIR/genome.gtf

--2020-06-25 12:35:33--  ftp://ftp.ensembl.org/pub/release-100/gtf/homo_sapiens/Homo_sapiens.GRCh38.100.gtf.gz
           => ‘Homo_sapiens.GRCh38.100.gtf.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-100/gtf/homo_sapiens ... done.
==> SIZE Homo_sapiens.GRCh38.100.gtf.gz ... 46973686
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh38.100.gtf.gz ... done.
Length: 46973686 (45M) (unauthoritative)

Homo_sapiens.GRCh38 100%[===================>]  44,80M  13,6MB/s    in 3,3s    

2020-06-25 12:35:36 (13,4 MB/s) - ‘Homo_sapiens.GRCh38.100.gtf.gz’ saved [46973686]



In [59]:
!./src/htseq.sh -d $WORKDIR -g $GENOMEDIR/genome.gtf

HTSeq is counting SRR6145423.sorted.bam.
100000 GFF lines processed.
200000 GFF lines processed.
300000 GFF lines processed.
400000 GFF lines processed.
500000 GFF lines processed.
600000 GFF lines processed.
700000 GFF lines processed.
800000 GFF lines processed.
900000 GFF lines processed.
1000000 GFF lines processed.
1100000 GFF lines processed.
1200000 GFF lines processed.
1300000 GFF lines processed.
1400000 GFF lines processed.
1500000 GFF lines processed.
1600000 GFF lines processed.
1700000 GFF lines processed.
1800000 GFF lines processed.
1900000 GFF lines processed.
2000000 GFF lines processed.
2100000 GFF lines processed.
2200000 GFF lines processed.
2300000 GFF lines processed.
2400000 GFF lines processed.
2500000 GFF lines processed.
2600000 GFF lines processed.
2700000 GFF lines processed.
2800000 GFF lines processed.
2900000 GFF lines processed.
2908224 GFF lines processed.
100000 SAM alignment records processed.
200000 SAM alignment records processed.
300000 SAM alignme

17800000 SAM alignment records processed.
17900000 SAM alignment records processed.
18000000 SAM alignment records processed.
18100000 SAM alignment records processed.
18200000 SAM alignment records processed.
18300000 SAM alignment records processed.
18400000 SAM alignment records processed.
18500000 SAM alignment records processed.
18600000 SAM alignment records processed.
18700000 SAM alignment records processed.
18800000 SAM alignment records processed.
18900000 SAM alignment records processed.
19000000 SAM alignment records processed.
19100000 SAM alignment records processed.
19200000 SAM alignment records processed.
19300000 SAM alignment records processed.
19400000 SAM alignment records processed.
19500000 SAM alignment records processed.
19600000 SAM alignment records processed.
19700000 SAM alignment records processed.
19800000 SAM alignment records processed.
19900000 SAM alignment records processed.
20000000 SAM alignment records processed.
20100000 SAM alignment records pro

37400000 SAM alignment records processed.
37500000 SAM alignment records processed.
37600000 SAM alignment records processed.
37700000 SAM alignment records processed.
37800000 SAM alignment records processed.
37900000 SAM alignment records processed.
38000000 SAM alignment records processed.
38100000 SAM alignment records processed.
38200000 SAM alignment records processed.
38300000 SAM alignment records processed.
38400000 SAM alignment records processed.
38500000 SAM alignment records processed.
38600000 SAM alignment records processed.
38700000 SAM alignment records processed.
38800000 SAM alignment records processed.
38900000 SAM alignment records processed.
39000000 SAM alignment records processed.
39100000 SAM alignment records processed.
39200000 SAM alignment records processed.
39300000 SAM alignment records processed.
39400000 SAM alignment records processed.
39500000 SAM alignment records processed.
39600000 SAM alignment records processed.
39700000 SAM alignment records pro

7500000 SAM alignment records processed.
7600000 SAM alignment records processed.
7700000 SAM alignment records processed.
7800000 SAM alignment records processed.
7900000 SAM alignment records processed.
8000000 SAM alignment records processed.
8100000 SAM alignment records processed.
8200000 SAM alignment records processed.
8300000 SAM alignment records processed.
8400000 SAM alignment records processed.
8500000 SAM alignment records processed.
8600000 SAM alignment records processed.
8700000 SAM alignment records processed.
8800000 SAM alignment records processed.
8900000 SAM alignment records processed.
9000000 SAM alignment records processed.
9100000 SAM alignment records processed.
9200000 SAM alignment records processed.
9300000 SAM alignment records processed.
9400000 SAM alignment records processed.
9500000 SAM alignment records processed.
9600000 SAM alignment records processed.
9700000 SAM alignment records processed.
9800000 SAM alignment records processed.
9900000 SAM alig

27100000 SAM alignment records processed.
27200000 SAM alignment records processed.
27300000 SAM alignment records processed.
27400000 SAM alignment records processed.
27500000 SAM alignment records processed.
27600000 SAM alignment records processed.
27700000 SAM alignment records processed.
27800000 SAM alignment records processed.
27900000 SAM alignment records processed.
28000000 SAM alignment records processed.
28100000 SAM alignment records processed.
28200000 SAM alignment records processed.
28300000 SAM alignment records processed.
28400000 SAM alignment records processed.
28500000 SAM alignment records processed.
28600000 SAM alignment records processed.
28700000 SAM alignment records processed.
28800000 SAM alignment records processed.
28900000 SAM alignment records processed.
29000000 SAM alignment records processed.
29100000 SAM alignment records processed.
29200000 SAM alignment records processed.
29300000 SAM alignment records processed.
29400000 SAM alignment records pro

...

18300000 SAM alignment records processed.
18400000 SAM alignment records processed.
18500000 SAM alignment records processed.
18600000 SAM alignment records processed.
18700000 SAM alignment records processed.
18800000 SAM alignment records processed.
18900000 SAM alignment records processed.
19000000 SAM alignment records processed.
19100000 SAM alignment records processed.
19200000 SAM alignment records processed.
19300000 SAM alignment records processed.
19400000 SAM alignment records processed.
19500000 SAM alignment records processed.
19600000 SAM alignment records processed.
19700000 SAM alignment records processed.
19800000 SAM alignment records processed.
19900000 SAM alignment records processed.
20000000 SAM alignment records processed.
20100000 SAM alignment records processed.
20200000 SAM alignment records processed.
20300000 SAM alignment records processed.
20400000 SAM alignment records processed.
20500000 SAM alignment records processed.
20600000 SAM alignment records pro

37900000 SAM alignment records processed.
38000000 SAM alignment records processed.
38100000 SAM alignment records processed.
38200000 SAM alignment records processed.
38300000 SAM alignment records processed.
38400000 SAM alignment records processed.
38500000 SAM alignment records processed.
38600000 SAM alignment records processed.
38700000 SAM alignment records processed.
38800000 SAM alignment records processed.
38900000 SAM alignment records processed.
39000000 SAM alignment records processed.
39100000 SAM alignment records processed.
39200000 SAM alignment records processed.
39300000 SAM alignment records processed.
39400000 SAM alignment records processed.
39500000 SAM alignment records processed.
39600000 SAM alignment records processed.
39700000 SAM alignment records processed.
39800000 SAM alignment records processed.
39900000 SAM alignment records processed.
40000000 SAM alignment records processed.
40100000 SAM alignment records processed.
40200000 SAM alignment records pro

6800000 SAM alignment records processed.
6900000 SAM alignment records processed.
7000000 SAM alignment records processed.
7100000 SAM alignment records processed.
7200000 SAM alignment records processed.
7300000 SAM alignment records processed.
7400000 SAM alignment records processed.
7500000 SAM alignment records processed.
7600000 SAM alignment records processed.
7700000 SAM alignment records processed.
7800000 SAM alignment records processed.
7900000 SAM alignment records processed.
8000000 SAM alignment records processed.
8100000 SAM alignment records processed.
8200000 SAM alignment records processed.
8300000 SAM alignment records processed.
8400000 SAM alignment records processed.
8500000 SAM alignment records processed.
8600000 SAM alignment records processed.
8700000 SAM alignment records processed.
8800000 SAM alignment records processed.
8900000 SAM alignment records processed.
9000000 SAM alignment records processed.
9100000 SAM alignment records processed.
9200000 SAM alig

26400000 SAM alignment records processed.
26500000 SAM alignment records processed.
26600000 SAM alignment records processed.
26700000 SAM alignment records processed.
26800000 SAM alignment records processed.
26900000 SAM alignment records processed.
27000000 SAM alignment records processed.
27100000 SAM alignment records processed.
27200000 SAM alignment records processed.
27300000 SAM alignment records processed.
27400000 SAM alignment records processed.
27500000 SAM alignment records processed.
27600000 SAM alignment records processed.
27700000 SAM alignment records processed.
27800000 SAM alignment records processed.
27900000 SAM alignment records processed.
28000000 SAM alignment records processed.
28100000 SAM alignment records processed.
28200000 SAM alignment records processed.
28300000 SAM alignment records processed.
28400000 SAM alignment records processed.
28500000 SAM alignment records processed.
28600000 SAM alignment records processed.
28700000 SAM alignment records pro

46000000 SAM alignment records processed.
46100000 SAM alignment records processed.
46200000 SAM alignment records processed.
46300000 SAM alignment records processed.
46400000 SAM alignment records processed.
46500000 SAM alignment records processed.
46600000 SAM alignment records processed.
46700000 SAM alignment records processed.
46800000 SAM alignment records processed.
46900000 SAM alignment records processed.
47000000 SAM alignment records processed.
47100000 SAM alignment records processed.
47200000 SAM alignment records processed.
47300000 SAM alignment records processed.
47400000 SAM alignment records processed.
47500000 SAM alignment records processed.
47600000 SAM alignment records processed.
47700000 SAM alignment records processed.
47800000 SAM alignment records processed.
47900000 SAM alignment records processed.
48000000 SAM alignment records processed.
48100000 SAM alignment records processed.
48200000 SAM alignment records processed.
48300000 SAM alignment records pro

I want to know if there is a difference in expression of these two samples.
DESeq2 is not possible for this, since there are only two samples.
Therefore, I am going to load the data in a pandas dataframe. Sort it for the hits, and see if it shows a difference.

In [71]:
workdir = os.environ['WORKDIR']

files = []

for file in os.listdir(workdir):
    if file.endswith('count_2.txt'):
        files.append(file)

['SRR6145423.sra_out.sam.sorted.bam_count_2.txt',
 'SRR6145433.sra_out.sam.sorted.bam_count_2.txt']

In [99]:
dataframes = {}

for file in files:
    SRA = file.split('.')[0]
    dataframes[SRA] = pandas.read_csv(os.path.join(workdir, file), sep="\t", header=None)
    dataframes[SRA] = dataframes[SRA][0:-5]
    dataframes[SRA] = dataframes[SRA].sort_values(by=1, axis=0, ascending=False)
    print(SRA)
    print(dataframes[SRA].head(10))
    

SRR6145423
                     0       1
53715  ENSG00000276168  410517
17859  ENSG00000198804  282358
21416  ENSG00000210082  252877
39793  ENSG00000251562  196757
17908  ENSG00000198886  165297
17847  ENSG00000198786  159133
17910  ENSG00000198888  108801
17818  ENSG00000198727   99492
17940  ENSG00000198938   98214
17810  ENSG00000198712   94738
SRR6145433
                     0       1
21416  ENSG00000210082  399734
53715  ENSG00000276168  374204
39793  ENSG00000251562  212113
21452  ENSG00000211459  146131
17859  ENSG00000198804  134142
17908  ENSG00000198886   91170
17847  ENSG00000198786   70780
6396   ENSG00000131095   61678
19684  ENSG00000203930   58354
17818  ENSG00000198727   57983
